- *FEA.Dev*                   
    https://www.linkedin.com/company/fea-dev/  
    https://github.com/fea-dev-usp

### Autores:

- *Gustavo Yuji Yamachi*      
    https://www.linkedin.com/in/gustavo-yuji-osawa-yamachi-4193371a1/  
    https://github.com/Gust4242

- *Kevyn Andrade Marcelino*   
    https://www.linkedin.com/in/kevyn-a-marcelino/  
    https://github.com/k-marcelino


Data: 2023-08-26

# -------------------------------------------------------------------------

### **IMPORTS**

In [ ]:
# DESCOMENTAR E COLOCAR O NOME DA LIB QUE QUER INSTALAR

# pip install sidrapy

In [ ]:
import zipfile
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from io import BytesIO
import requests
import sidrapy
import wget
import json
from urllib.request import urlopen

### **Obtendo tabela SIDRAPY**


In [ ]:
# Tabelas Censos - https://sidra.ibge.gov.br/pesquisa/censo-demografico/demografico-2022/primeiros-resultados-populacao-e-domicilios
# Parâmetros da API -> Consulta: https://apisidra.ibge.gov.br/

pop = sidrapy.get_table(
    table_code='4709',              # t (table_code) - é o código da tabela referente ao indicador e a pesquisa;
    territorial_level='6',          # n (territorial_level) - especifica os níveis territoriais;
    ibge_territorial_code='all',    # n/ (ibge_territorial_code) - inserido dentro do nível territorial, especificar o código territorial do IBGE;
    period='all',                   # p (period) - utilizado para especificar o período;
    variable='all',                 # v (variable) - para especificar as variáveis desejadas;
    #header='y'                     # h (header) - para especificar se o cabeçalho será retornado ou não -> Bom para saber o que cada coluna significa
)

print(pop.shape)
pop.head()

### Tratamentos

In [ ]:
# Removendo acentos
pop = pop.apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8') if x.dtype == 'object' else x) 
# Removendo primeira linha
pop = pop.iloc[1:]

pop.head()

In [ ]:
# Pivotando tabela
pop = pop.pivot_table(index=['D1C', 'D1N'], columns='D3N', values='V').reset_index()
pop.columns.name = None

print(pop.shape)
pop.head()

In [ ]:
# Dict mapeamento nomes das colunas
column_mapping = {
    'D1C': 'CD_MUN',
    'D1N': 'NM_MUN',
    'Populacao residente': 'POP_RESIDENTE',
    'Variacao absoluta da populacao residente 2010 compatibilizada': 'VAR_ABS',
    'Taxa de crescimento geometrico': 'TX_CRESC'
}

# Renomeando colunas
pop.rename(columns=column_mapping, inplace=True)

print(pop.shape)
pop.head()

In [ ]:
pop.info()

In [ ]:
# Tratando tipos das colunas
pop[['CD_MUN', 'POP_RESIDENTE', 'VAR_ABS']] = pop[['CD_MUN', 'POP_RESIDENTE', 'VAR_ABS']].astype(int)
pop['TX_CRESC'] = pop['TX_CRESC'].astype(float) #/ 100  # Deixar como porcentagem por enquanto, fica mais fácil para visualizar no mapa

pop.head()

In [ ]:
# Salvando arquivo
pop.to_excel('pop.xlsx', index=False, freeze_panes=(1,1))

### **Obtendo Arquivo GeoJson**

#### Alternativa 1

    Baixar arquivo json e ler localmente

In [ ]:
# Descomentar e rodar caso quiser essa aternativa

# geo_json_url = 'https://github.com/fea-dev-usp/IBGE/blob/master/geojson_2022.json'
# wget.download(geo_json_url)

# with open('geo-ibge.json', 'r', encoding='utf-8') as file:
#     geo_json = json.load(file)

# geo_json

#### Alternativa 2

    Ler arquivo json direto do github fea.dev

In [ ]:
with urlopen('https://raw.githubusercontent.com/fea-dev-usp/IBGE/master/geo-ibge.json') as response:
    geo_json = json.load(response)

geo_json["features"][0]

In [ ]:
with open('geojson_2022.json', 'r', encoding='utf-8') as file:
    geo_json = json.load(file)

geo_json

### **Plotando Mapa - Plotly**

In [ ]:
import plotly.express as px

# Create the choropleth map using Plotly Express
fig = px.choropleth_mapbox(
    data_frame=pop,
    geojson=geo_json,
    locations='CD_MUN',
    featureidkey='properties.codarea',
    color='TX_CRESC',
    color_continuous_scale='thermal',
    range_color=(pop['TX_CRESC'].min(), pop['TX_CRESC'].max()),
    mapbox_style='open-street-map',
    zoom=3.5,
    center={"lat": -15.81, "lon": -47.93},
    opacity=1,
    labels={'TX_CRESC': 'Tx Cresc Geométrico (%)', 'CD_MUN': 'Código do município'},
    width=1200,
    height=800,
    title='Variação Pop Censo 2022',
    hover_name='NM_MUN'  # Add this line to set hover name
    #hover_data={'CD_MUN': False, 'TX_CRESC': True}  # Include only desired hover data

)

# Customize layout
fig.update_layout(
    margin={'r': 0, 't': 1, 'l': 0, 'b': 1},
    coloraxis_colorbar={
        'title': {
            'text': 'Taxa de Crescimento (%)',
            'side': 'right'
        }
    }
)

# Customize the individual polygons
fig.update_geos(
    showcoastlines=False,  # Hide coastlines
    visible=False  # Hide geographical boundaries
)

# Update marker settings
fig.update_traces(
    marker_line_width=0.05,
    selector=dict(type='choroplethmapbox')
)

# Display the map
fig.show()


In [ ]:
pop.sort_values('TX_CRESC', ascending=False)

In [ ]:
import plotly.express as px
from ipywidgets import interact, widgets
from IPython.display import display



# Create the choropleth map using Plotly Express
def create_choropleth(selected_variable):
    fig = px.choropleth_mapbox(
        data_frame=pop,
        geojson=geo_json,
        locations='CD_MUN',
        featureidkey='properties.codarea',
        color=selected_variable,
        color_continuous_scale='thermal',
        range_color=(pop[selected_variable].min(), pop[selected_variable].max()),
        mapbox_style='open-street-map',
        zoom=3.5,
        center={"lat": -15.81, "lon": -47.93},
        opacity=1,
        hover_name='NM_MUN',
        hover_data={'CD_MUN': False, selected_variable: True}
    )
    
    # Customize layout and traces...
    
    fig.show()

# Define a dropdown widget with the available options
dropdown = widgets.Dropdown(
    options=[
        ('Taxa de Crescimento Geométrico', 'TX_CRESC'),
        ('População Residente', 'POP_RESIDENTE')
    ],
    value='TX_CRESC',  # Initial value
    description='Select Variable:'
)

# Define the interactive function using the interact function
def interactive_choropleth(selected_variable):
    create_choropleth(selected_variable)

# Link the dropdown to the interactive function
interact(interactive_choropleth, selected_variable=dropdown)

# Display the dropdown widget
display(dropdown)
